## Bin Packing
Задача решается при условии наличии решения задачи именно целочисленной <br>
Суть простая. Пытаемся решить задачу для конкретного числа рюкзаков --- если решение существует, то выдаём его. Если нет --- увеличиваем количество рюкзаков на единицу.

In [ ]:
from cvxopt import matrix, solvers
import numpy as np
w = [6.0, 6.0, 6.0, 7.0, 7.0, 7.0, 8.0, 8.0, 8.0, 9.0, 9.0, 9.0, 10.0, 10.0, 10.0]

for k in range(1, 16):
    Ax = [[0 for j in range(k * 16 + 30)] for i in range(15 * k)]
    c = matrix([0.0 for i in range(15 * k)])
    for i in range(15 * k):
        Ax[i][i // 15] = w[i % 15]
    for i in range(15 * k):
        Ax[i][i + k] = -1.0
    for i in range(15):
        for j in range(k):
            Ax[15 * j + i][16 * k + i] = 1.0
    for i in range(15):
        for j in range(k):
            Ax[15 * j + i][16 * k + 15 + i] = -1.0
    A = matrix(Ax)
    b = matrix([20.0 for i in range(k)] + [0.0 for i in range(15*k)] + [1.0 for i in range(15)] + [-1.0 for i in range(15)])
    sol=solvers.lp(c,A,b)
    if sol['x'] == None:
        continue
    else:
        print(sol['x'])
        print(k)
        break

## Coals
Это совершенно базовая задача на ограничения. Перебираем все наборы из любых $4$ компонент и решаем обычную линейную задачу для неё. Из получившихся ответов ищем минимум

In [ ]:
from cvxopt import matrix, solvers
import numpy as np
import sys
sys.setrecursionlimit(100000)

def GetAllCombinations(comb, k):
    if (k == 4):
        allCombs.append(comb)
        return
    start = 0
    if (len(comb) != 0):
        start = comb[-1]
    for i in range(start + 1, 6 + k):
        GetAllCombinations(comb + [i], k + 1)

allCombs = []
GetAllCombinations([], 0)
price = [0.0, 12.0, 14.0, 17.0, 10.0, 13.0, 9.0, 15.0, 11.0]
prop = [0.0, 0.02, 0.025, 0.01, 0.05, 0.01, 0.05, 0.02, 0.01]
minimum = 1000000.0
solution = []
for comb in allCombs:
    A = matrix([[-1.0, 0.0, 0.0, 0.0, -19*prop[comb[0]], prop[comb[0]], prop[comb[0]], prop[comb[0]], prop[comb[0]] - 1.8],
                [0.0, -1.0, 0.0, 0.0, prop[comb[1]], -19*prop[comb[1]], prop[comb[1]], prop[comb[1]], prop[comb[1]] - 1.8],
                [0.0, 0.0, -1.0, 0.0, prop[comb[2]], prop[comb[2]], -19*prop[comb[2]], prop[comb[2]], prop[comb[2]] - 1.8],
                [0.0, 0.0, 0.0, -1.0, prop[comb[3]], prop[comb[3]], prop[comb[3]], -19*prop[comb[3]], prop[comb[3]] - 1.8]])
    b = matrix([0.0 for i in range(9)])
    c = matrix([price[comb[i]] for i in range(4)])
    sol=solvers.lp(c,A,b)
    sol = sol['x']
    if (sol != None):
        summ = 0
        for i in range(len(c)):
            summ += c[i] * sol[i]
        if (summ < minimum):
            minimum = summ
            solution = comb
            
print("OPTIMAL\n")
print(solution)
print(sum([prop[solution[i]] for i in range(len(solution))]) / 4)

## Independent Set (Stable)
В этой задаче всё не так уж и просто. Пришлось придумывать нормальные ограничения. <br>
Заметим, что если каждой включённой вершине поставить в соответствие $-1$, а каждому включённому ребру --- $2n$, то минимизация $\sum_{vert \in G}(-1) + \sum_{edge \in G}2n$ Будет как раз эквивалентна нахождению задаче ахождения независимого множества (потому что любой ответ, содержащий ребро, даёт больше число, нежели ответ из одной вершины, поэтому множество независимо. Ну и максимально, потому что выгодно взять сумму $-1$ как можно большей). <br>
Заметим также, что в задаче возникает условия следующего типа: $$0 \leqslant y[i][j] \leqslant adj[i][j]$$ $$y[i][j] \leqslant x[i]x[j]$$
где $x$ --- переменные, индикаторы того, что вершина взята в рассмотрение, а $y[i][j]$ --- индикатор того, что ребро из $i$ в $j$ взято в рассмотрение. Второе условие, оказывается, при данных целочисленных ограничениях можно будет переписать в виде: $$y[i][j] \geqslant x[i] + x[j] - 1$$, при этом ограничения будут рассматриваться только для рёбер, что $adj[i][j] = 1$

In [ ]:
from cvxopt import matrix, solvers
import numpy as np
n = 4;
adj = [[0, 1, 1, 1],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 0, 0, 0]];
# n = 2
# adj = [[0.0, 0.0],
#        [0.0, 0.0]]

k = 0
for i in range(n):
    for j in range(i, n):
        k += int(adj[i][j])
print(k)


c = matrix([-1.0 for i in range(n)] + [2.0 * n for i in range(n ** 2)])
Ax = [[0.0 for i in range(2 * n ** 2 + 2 * n + k)] for j in range(n + n ** 2)]
b = []
for i in range(n):
    Ax[i][i] = 1.0
    b.append(1.0)
for i in range(n):
    Ax[i][i + n] = -1.0
    b.append(0.0)
for i in range(n ** 2):
    Ax[i + n][i + 2 * n] = 1.0
    b.append(adj[i // n][i % n])
for i in range(n ** 2):
    Ax[i + n][i + 2 * n + n ** 2] = -1.0
    b.append(0.0)
counter = 0
for i in range(n):
    for j in range(i, n):
        if (adj[i][j] == 1.0):
            Ax[i][counter + 2 * n ** 2 + 2 * n] = 1.0
            Ax[j][counter + 2 * n ** 2 + 2 * n] = 1.0
            Ax[n + i * n + j][counter + 2 * n ** 2 + 2 * n] = -1.0
            b.append(1.0)
            counter += 1
# for i in Ax:
#     print(i)
# print(b)
A = matrix(Ax)
b = matrix(b)
sol=solvers.lp(c,A,b)
sol = sol['x'][:n]
print(sol)
print("Количество вершин -", int(np.round(sum(sol))))

## Maximum Matching
Идея проста. Надо сказать, что переменные соответствуют занятости рёбер. Если $0$ --- мы конкретное ребро включаем, а если $1$ --- не включаем. <br>
Заметим, что ограничения видна $\sum a_{ij} \leqslant 1$ как раз задают условие, что нет двух рёбер включённых, инцидентных $i$-ой вершине. Поэтому оно как раз задаёт, что мы выбираем независимое множество рёбер. <br>
Остальные ограничения просто задают то, что наш набор чисел согласуется с матрицей смежности

In [ ]:
from cvxopt import matrix, solvers
import numpy as np
n = 4
adj  = [[0.0, 1.0, 1.0, 1.0],
        [1.0, 0.0, 1.0, 1.0],
        [1.0, 1.0, 0.0, 0.0],
        [1.0, 1.0, 0.0, 0.0]]
w    = [[0.0, 1.0, 1.0, 2.0],
        [1.0, 0.0, 2.0, 1.0],
        [1.0, 2.0, 0.0, 0.0],
        [2.0, 1.0, 0.0, 0.0]]


# n = 2
# adj  = [[0.0, 1.0],
#         [1.0, 0.0]]
# w    = [[0.0, 1.0],
#         [1.0, 0.0]]

eps = 1e-4


c = [0.0 for i in range(n ** 2)]
for i in range(n):
    for j in range(n):
        c[i * n + j] = -w[i][j]
c = matrix(c)
Ax = [[0.0 for i in range(2 * n ** 2 + n)] for j in range(n ** 2)]
b = []
for i in range(n ** 2):
    Ax[i][i] = 1.0
    b.append(adj[i // n][i % n])
for i in range(n ** 2):
    Ax[i][i + n ** 2] = -1.0
    b.append(0.0)
for i in range(n):
    for j in range(n):
        Ax[i * n + j][i + 2 * n ** 2] = 1.0
    b.append(1.0)
# for i in Ax:
#     print(i)
# print(b)
A = matrix(Ax)
b = matrix(b)
sol=solvers.lp(c,A,b)
sol = sol['x']
for idx in range(len(sol)):
    if (abs(sol[idx] - 1.0) < eps):
        i = idx // n + 1
        j = idx % n + 1
        if (i < j):
            print(i, "---", j)
sum_w = 0
for i in range(n ** 2):
    sum_w += sol[i] * w[i // n][i % n]
print(np.round(sum_w / 2))

## Chromatic Number
Идея простая --- мы сопоставляем каждому $k$-ому цвету, соответственно, число $n^k$. Тогда очевидно, поскольку всего вершин $n$ --- эта сумма при меньшем количестве цветов будет меньше, чем при большем. Поэтому минимизация суммы чисел, соответствующих каждой вершине, соответствует нахождению хроматического числа. <br>
По числу просто понять, сколько цветов --- $sum // n$ пока не получился $0$ даст количество цветов. <br>